In [55]:
import numpy as np
from typing import Tuple
from enum import Enum
import math

In [50]:
class Pivoting(Enum):
  NO_PIVOTING = 0,
  PARTIAL_PIVOTING = 1,
  FULL_PIVOTING = 2

def do_partial_pivoting(A: np.ndarray, b:np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  # find hihest row value
  index_highest_values_per_column = np.abs(A).argmax(1)
  index_hihest_first_column = index_highest_values_per_column[0]

  # Swap rows
  A[[0, index_hihest_first_column]] = A[[index_hihest_first_column, 0]]
  b[[0, index_hihest_first_column]] = b[[index_hihest_first_column, 0]]

  return (A, b)

def do_partial_pivoting_lu(upper : np.ndarray, lower : np.ndarray, b : np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  index_highest_values_per_column = np.abs(upper).argmax(1)
  index_hihest_first_column = index_highest_values_per_column[0]
  
  # Swap rows
  upper[[0, index_hihest_first_column]] = upper[[index_hihest_first_column, 0]]
  # lower[[0, index_hihest_first_column]] = lower[[index_hihest_first_column, 0]]
  b[[0, index_hihest_first_column]] = b[[index_hihest_first_column, 0]]

  return (upper, lower, b)


def do_full_pivoting(A : np.ndarray, b:np.ndarray) -> Tuple[np.ndarray, np.ndarray, int]:
  # Find the highest value
  highest_value = max(A.min(), A.max(), key=abs)
  max_value = np.where(A == highest_value)

  # Use the first found occurence if there is more than one
  index_row_highest_value = max_value[0][0]
  index_column_highest_value = max_value[1][0]

  # Swap rows
  A[[0, index_row_highest_value]] = A[[index_row_highest_value, 0]]
  b[[0, index_row_highest_value]] = b[[index_row_highest_value, 0]]

  # Swap columns
  A[:, [0, index_column_highest_value]] = A[:, [index_column_highest_value, 0]]

  return (A, b, index_column_highest_value)

# Gaussian Elimination method

This method for solving linear systems works by forming a superior triangle matrix 

In [4]:
def gauss_elimination(A: np.ndarray, b:np.ndarray, pivoting = Pivoting.NO_PIVOTING) -> np.ndarray:
  assert len(A) == len(b)
  assert A.shape[0] == A.shape[1]
  exchanged_columns = []

  for index_col in range(len(A)):
    match pivoting:
      case Pivoting.NO_PIVOTING:
        pass
      case Pivoting.PARTIAL_PIVOTING:
        A[index_col:, index_col:] , b[index_col:] = do_partial_pivoting(A[index_col:, index_col:] , b[index_col:])
      case Pivoting.FULL_PIVOTING:
        A[index_col:, index_col:], b[index_col:], local_column_excanged = do_full_pivoting(A[index_col:, index_col:] , b[index_col:])
        exchanged_columns.append(local_column_excanged)
      
    pivot = A[index_col][index_col]
    for index_row_below in range(index_col + 1, len(A)):
      ratio = A[index_row_below][index_col] / pivot
      
      for idx in range(index_col, len(A)):
        A[index_row_below][idx] -= A[index_col][idx] * ratio

      b[index_row_below] -= b[index_col] * ratio
      

  result = np.linalg.solve(A, b)
  if pivoting == Pivoting.FULL_PIVOTING:
    for index, excahnged in enumerate(exchanged_columns):
      result[[index, index + excahnged]] = result[[index + excahnged, index]]

  return result

# LU Factoring Method

In [51]:
def is_possible_to_apply_LU_method(A:np.ndarray) -> bool:
  if A.ndim == 2 and A.shape[0] == A.shape[1]: 
    if A.shape == (0,0):
      return True
    return np.linalg.det(A) != 0 and is_possible_to_apply_LU_method(A[1:, 1:])
  return False

def lu_factoring(A: np.ndarray, b:np.ndarray, pivoting = Pivoting.NO_PIVOTING) -> np.ndarray:
  assert len(A) == len(b)
  assert A.shape[0] == A.shape[1]
  assert is_possible_to_apply_LU_method(A) == True

  lower_matrix = np.identity(len(A), np.float64)
  upper_matrix = A

  for index_col in range(len(upper_matrix)):
    match pivoting:
      case Pivoting.NO_PIVOTING:
        pass
      case Pivoting.PARTIAL_PIVOTING:
        upper_matrix[index_col:, index_col:], lower_matrix[index_col:, index_col:], b[index_col:] = do_partial_pivoting_lu(upper_matrix[index_col:, index_col:], lower_matrix[index_col:, index_col:], b[index_col:])
      case Pivoting.FULL_PIVOTING:
        raise NotImplemented("Not implemented for LU")
      
    pivot = upper_matrix[index_col, index_col]
    
    for index_row_below in range(index_col + 1, len(upper_matrix)):
      ratio = upper_matrix[index_row_below, index_col]/pivot
      lower_matrix[index_row_below, index_col] = ratio

      for idx in range(index_col, len(A)):
        upper_matrix[index_row_below, idx] -= upper_matrix[index_col, idx] * ratio

  partial_result = np.linalg.solve(lower_matrix, b)
  result = np.linalg.solve(upper_matrix, partial_result)
  return result

# Cholesky method

In [54]:
def is_possible_to_apply_Cholesky_method(A:np.ndarray) -> bool:
  if A.ndim == 2 and A.shape[0] == A.shape[1]: 
    if A.shape == (0,0):
      return True
    return np.linalg.det(A) > 0.0 and np.allclose(A, A.T) and is_possible_to_apply_LU_method(A[1:, 1:])
  return False

def cholesky_method(A:np.ndarray, b:np.ndarray) -> np.ndarray:
  assert len(A) == len(b)
  assert A.shape[0] == A.shape[1]
  assert is_possible_to_apply_Cholesky_method(A) == True

  G = np.zeros(A.shape)

  for index_row in len(A):
    # G[index_row, 0] = 0
  

  

# Gauss Jacobi

In [ ]:
def is_possible_to_apply_Jacobi_method(A:np.ndarray) -> bool:
  pass

def jacobi_method(A:np.ndarray) -> np.ndarray:
  pass

# Gaus Seidel

In [ ]:
def is_possible_to_apply_Seidel_method(A:np.ndarray) -> bool:
  pass

def seidel_method(A:np.ndarray) -> np.ndarray:
  pass

# Gauss with pivoting

In [31]:
A = np.array([[1,-7], [5,2]], dtype=np.float64)
b = np.array([-11,-18], dtype=np.float64)
# Solution x=-4, y=1
gauss_elimination(A,b, Pivoting.PARTIAL_PIVOTING)

array([-4.,  1.])

In [53]:
A = np.array([
  [3,2,4],
  [1,1,2],
  [4,3,-2],], 
dtype=np.float64)
b = np.array([1,2,3], np.float64)

# {-3, 5, 0}
lu_factoring(A, b, Pivoting.PARTIAL_PIVOTING)

array([ 1.5, -1. ,  0. ])

In [73]:
# 1x + 2y - 2z = -15
# 2x + 1y - 5z = -21
# 1x - 4y + 1z = +18 
# Solution {-1, -4, 3}

A = np.array([
  [1, 2, -2],
  [2, 1, -5],
  [1, -4, 1]
], dtype=np.float64)

b = np.array([-15, -21, 18], dtype=np.float64)

gauss_elimination(A, b, Pivoting.FULL_PIVOTING)

array([ 3., -4., -1.])

In [82]:
A = np.array([
  [1.,    2.,     1., -1.],
  [3./2., 1.,     2., 2.],
  [4.,    4.,     3., 4.],
  [2./5., 0.,  1./5., 1.]
], dtype=np.float64)

b = np.array([5., 8., 22., 3.], dtype=np.float64)

# x = 16
# y = -6
# z = -2
# t = -3

gauss_elimination(A, b, Pivoting.FULL_PIVOTING)

index 0, exchanged: 0
index 1, exchanged: 2
index 2, exchanged: 0
index 3, exchanged: 0


array([16., -6., -2., -3.])